# ExampleGen Test run

## Set up

TFX requires apache-airflow and docker SDK.

In [1]:
!pip install 'apache-airflow[gcp]' docker tfx

You should consider upgrading via the 'pip install --upgrade pip' command.


In this notebook, we use TFX version 0.13.0

In [2]:
import tfx
tfx.version.__version__

'0.13.0'

TFX requires TensorFlow >= 1.13.1

In [3]:
import tensorflow as tf
tf.__version__

'1.13.1'

TFX supports Python 3.5 from version 0.13.0

In [4]:
import sys
sys.version

'3.5.2 (default, Nov 12 2018, 13:43:14) \n[GCC 5.4.0 20160609]'

## Download sample data

In [5]:
%%bash
# This enables you to run this notebook twice.
# There should not be train/eval files at ~/taxi/data, since TFX can handle only single file with version 0.13.0
if [ -e ~/taxi/data ]; then
    rm -rf ~/taxi/data
fi

# download taxi data
mkdir -p ~/taxi/data/simple
mkdir -p ~/taxi/serving_model/taxi_simple
wget https://raw.githubusercontent.com/tensorflow/tfx/master/tfx/examples/chicago_taxi_pipeline/data/simple/data.csv -O ~/taxi/data/simple/data.csv

# download 
wget https://raw.githubusercontent.com/tensorflow/tfx/master/tfx/examples/chicago_taxi_pipeline/taxi_utils.py -O ~/taxi/taxi_utils.py

--2019-06-12 09:28:01--  https://raw.githubusercontent.com/tensorflow/tfx/master/tfx/examples/chicago_taxi_pipeline/data/simple/data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.108.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1922668 (1.8M) [text/plain]
Saving to: ‘/root/taxi/data/simple/data.csv’

     0K .......... .......... .......... .......... ..........  2% 3.26M 1s
    50K .......... .......... .......... .......... ..........  5% 6.90M 0s
   100K .......... .......... .......... .......... ..........  7% 22.9M 0s
   150K .......... .......... .......... .......... .......... 10% 24.0M 0s
   200K .......... .......... .......... .......... .......... 13% 3.58M 0s
   250K .......... .......... .......... .......... .......... 15% 21.9M 0s
   300K .......... .......... .......... .......... .......... 18% 40.8M 0s
   350K ........

## Import

In [6]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import datetime
import logging
import os
from google.protobuf import json_format

from tfx.components.base.base_component import ComponentOutputs
from tfx.components.evaluator.component import Evaluator
from tfx.components.example_gen.csv_example_gen.component import CsvExampleGen
from tfx.components.example_validator.component import ExampleValidator
from tfx.components.model_validator.component import ModelValidator
from tfx.components.pusher.component import Pusher
from tfx.components.schema_gen.component import SchemaGen
from tfx.components.statistics_gen.component import StatisticsGen
from tfx.components.trainer.component import Trainer
from tfx.components.transform.component import Transform
from tfx.orchestration import pipeline
from tfx.orchestration.airflow.airflow_runner import AirflowDAGRunner
from tfx.proto import evaluator_pb2
from tfx.proto import example_gen_pb2
from tfx.proto import pusher_pb2
from tfx.proto import trainer_pb2
from tfx.utils.dsl_utils import csv_input
from tfx.utils.channel import Channel
from tfx.utils import types

/usr/local/lib/python3.5/dist-packages/apache_beam/__init__.py:84: UserWarning: Running the Apache Beam SDK on Python 3 is not yet fully supported. You may encounter buggy behavior or missing features.
  'Running the Apache Beam SDK on Python 3 is not yet fully supported. '


## configs

In [7]:
# This example assumes that the taxi data is stored in ~/taxi/data and the
# taxi utility function is in ~/taxi.  Feel free to customize this as needed.
_taxi_root = os.path.join(os.environ['HOME'], 'taxi')
_data_root = os.path.join(_taxi_root, 'data/simple')
# Python module file to inject customized logic into the TFX components. The
# Transform and Trainer both require user-defined functions to run successfully.
_taxi_module_file = os.path.join(_taxi_root, 'taxi_utils.py')
# Path which can be listened to by the model server.  Pusher will output the
# trained model here.
_serving_model_dir = os.path.join(_taxi_root, 'serving_model/taxi_simple')

# Directory and data locations.  This example assumes all of the chicago taxi
# example code and metadata library is relative to $HOME, but you can store
# these files anywhere on your local filesystem.
_tfx_root = os.path.join(os.environ['HOME'], 'tfx')
_pipeline_root = os.path.join(_tfx_root, 'pipelines')
_metadata_db_root = os.path.join(_tfx_root, 'metadata')
_log_root = os.path.join(_tfx_root, 'logs')

# Airflow-specific configs; these will be passed directly to airflow
_airflow_config = {
    'schedule_interval': None,
    'start_date': datetime.datetime(2019, 1, 1),
}

# Logging overrides
logger_overrides = {'log_root': _log_root, 'log_level': logging.INFO}

## Create ExampleGen

In [8]:
"""Implements the chicago taxi pipeline with TFX."""
examples = csv_input(_data_root)

# Brings data into the pipeline or otherwise joins/converts training data.
train_config = example_gen_pb2.SplitConfig.Split(name='train', hash_buckets=2)
eval_config = example_gen_pb2.SplitConfig.Split(name='eval', hash_buckets=1)
output_config = example_gen_pb2.Output(
    split_config=example_gen_pb2.SplitConfig(splits=[
        train_config,
        eval_config
    ]))

train_examples = types.TfxType(type_name='ExamplesPath', split='train')
train_examples.uri = os.path.join(_data_root, 'train')
eval_examples = types.TfxType(type_name='ExamplesPath', split='eval')
eval_examples.uri = os.path.join(_data_root, 'eval')
output_dict = {'examples': Channel(
    type_name='ExamplesPath',
    static_artifact_collection=[train_examples, eval_examples])}
outputs = ComponentOutputs(output_dict)

example_gen = CsvExampleGen(input_base=examples, output_config=output_config, outputs=outputs)

## Execute

In [9]:
input_dict = {key:value.get() for key, value in example_gen.input_dict.items()}
output_dict = {key: value.get() for key, value in example_gen.outputs.get_all().items()}

exec_properties = {
    'output': json_format.MessageToJson(output_config)}

example_executor = example_gen.executor()
example_executor.Do(input_dict=input_dict, output_dict=output_dict, exec_properties=exec_properties)

INFO:tensorflow:Starting Executor execution.
[2019-06-12 09:28:04,045] {base_executor.py:72} INFO - Starting Executor execution.
INFO:tensorflow:Inputs for Executor is: {"input-base": [{"artifact": {"uri": "/root/taxi/data/simple", "properties": {"type_name": {"stringValue": "ExternalPath"}, "split": {"stringValue": ""}}}, "artifact_type": {"properties": {"type_name": "STRING", "span": "INT", "split": "STRING", "name": "STRING", "state": "STRING"}, "name": "ExternalPath"}}]}
[2019-06-12 09:28:04,054] {base_executor.py:74} INFO - Inputs for Executor is: {"input-base": [{"artifact": {"uri": "/root/taxi/data/simple", "properties": {"type_name": {"stringValue": "ExternalPath"}, "split": {"stringValue": ""}}}, "artifact_type": {"properties": {"type_name": "STRING", "span": "INT", "split": "STRING", "name": "STRING", "state": "STRING"}, "name": "ExternalPath"}}]}
INFO:tensorflow:Outputs for Executor is: {"examples": [{"artifact": {"uri": "/root/taxi/data/simple/train", "properties": {"type_n

[2019-06-12 09:28:13,932] {fn_api_runner.py:437} INFO - Running ((ref_PCollection_PCollection_24/Read)+(ref_AppliedPTransform_OutputSplittrain/Write/WriteImpl/PreFinalize_50))+(ref_PCollection_PCollection_33/Write)
[2019-06-12 09:28:13,961] {fn_api_runner.py:437} INFO - Running (((ref_AppliedPTransform_OutputSpliteval/Write/WriteImpl/DoOnce/Read_66)+(ref_AppliedPTransform_OutputSpliteval/Write/WriteImpl/InitializeWrite_67))+(ref_PCollection_PCollection_42/Write))+(ref_PCollection_PCollection_43/Write)
[2019-06-12 09:28:13,980] {fn_api_runner.py:437} INFO - Running (((ShuffleSpliteval/ReshufflePerKey/GroupByKey/Read)+((ref_AppliedPTransform_ShuffleSpliteval/ReshufflePerKey/FlatMap(restore_timestamps)_60)+(ref_AppliedPTransform_ShuffleSpliteval/RemoveRandomKeys_61)))+((ref_AppliedPTransform_OutputSpliteval/Write/WriteImpl/WriteBundles_68)+((ref_AppliedPTransform_OutputSpliteval/Write/WriteImpl/Pair_69)+(ref_AppliedPTransform_OutputSpliteval/Write/WriteImpl/WindowInto(WindowIntoFn)_70))))

## Check result



In [10]:
!ls -Rlhs /root/taxi/data/simple/

/root/taxi/data/simple/:
total 1.9M
1.9M -rw-r--r-- 1 root root 1.9M Jun 12 09:28 data.csv
4.0K drwxr-xr-x 2 root root 4.0K Jun 12 09:28 eval
4.0K drwxr-xr-x 2 root root 4.0K Jun 12 09:28 train

/root/taxi/data/simple/eval:
total 204K
204K -rw-r--r-- 1 root root 201K Jun 12 09:28 data_tfrecord-00000-of-00001.gz

/root/taxi/data/simple/train:
total 408K
408K -rw-r--r-- 1 root root 405K Jun 12 09:28 data_tfrecord-00000-of-00001.gz
